**Note:** This project contains setting up an appropriate directory structure which includes testing using pytest. Since the focus of these notebooks is the code, everything will be available in this one notebook under different headings. 

I have also cut out all docstrings for brevity.

### Project 3 - Single Inheritance - Solution

You are writing an inventory application for a budding tech guy who has a video channel featuring computer builds.
Basically they have a pool of inventory, (for example 5 x AMD Ryzen 2-2700 CPUs) that they use for builds. When they take a CPU from the pool, they will indicate this using the object that tracks that sepcific type of CPU. They may also purchase additional CPUs, or retire some (because they overclocked it too much and burnt them out!).

Technically we would want a database to back all this data, but here we're just going to build the classes we'll use while our program is running and not worry about retrieving or saving the state of the inventory.

The base class is going to be a general `Resource`. This class should provide functionality common to all the actual resources (CPU, GPU, Memory, HDD, SSD) - for this exercise we're only going to implement CPU, HDD and SSD.

It should provide this at a minimum:

- `name` : user-friendly name of resource instance (e.g.` Intel Core i9-9900K`)
- `manufacturer` - resource instance manufacturer (e.g. `Nvidia`)
- `total` : inventory total (how many are in the inventory pool)
- `allocated` : number allocated (how many are already in use)
- a `__str__` representation that just returns the resource name
- a mode detailed `__repr__` implementation
- `claim(n)` : method to take n resources from the pool (as long as inventory is available)
- `freeup(n)` : method to return n resources to the pool (e.g. disassembled some builds)
- `died(n)` : method to return and permanently remove inventory from the pool (e.g. they broke something) - as long as total available allows it
- `purchased(n)` - method to add inventory to the pool (e.g. they purchased a new CPU)
- `category` - computed property that returns a lower case version of the class name

Next we are going to define child classes for each of CPU, HDD and SDD.

For the `CPU` class:
- `cores` (e.g. `8`)
- `socket` (e.g. `AM4`)
- `power_watts` (e.g. `94`)

For the HDD and SDD classes, we're going to create an intermediate class called `Storage` with these additional properties:
- `capacity_GB` (e.g. `120`)

The `HDD` class extends `Storage` and has these additional properties:
- `size` (e.g. ``2.5"``)
- `rpm` (e.g. `7000`)

The `SSD` class extends `Storage` and has these additional properties:
- `interface` (e.g. `PCIe NVMe 3.0 x4`)

For all your classes, implement a full constructor that can be used to initialize all the properties, some form of validation on numeric types, as well as customized `__repr__` as you see fit.

For the `total` and `allocated` values in the `Resource` init, think of the arguments there as the **current** total and allocated counts. Those `total` and `allocated` attributes should be private **read-only** properties, but they are modifiable through the various methods such as `claim`, `return`, `died` and `purchased`. Other attributes like `name`, `manufacturer_name`, etc should be read-only.

# Solution

#### Main Solution

1. Create a helper function `validate_integer` that will allow us to validate that a value is an integer, optionally between a min and max (inclusive), and raises a `TypeError`, `ValueError` with a custom error message that can be overriden when bound checks fail.

2. Create the unit tests for the `validate_integer` function. (See "Tests Solution" subsection.)

3. Implement the `Resource` class.

4. Create the unit tests for the `Resource` class.

5. Create the `CPU`, `Storage`, `HDD` and `SSD` class.

6. Create the tests for the aforementioned classes.

In [1]:
# STEP 1
def validate_integer(
        arg_name, arg_value, min_value=None, max_value=None,
        custom_min_message=None, custom_max_message=None
):
    if not isinstance(arg_value, int):
        raise TypeError(f'{arg_name} must be an integer.')

    if min_value is not None and arg_value < min_value:
        if custom_min_message is not None:
            raise ValueError(custom_min_message)
        raise ValueError(f'{arg_name} cannot be less than {min_value}')

    if max_value is not None and arg_value > max_value:
        if custom_max_message is not None:
            raise ValueError(custom_max_message)
        raise ValueError(f'{arg_name} cannot be greater than {max_value}')

In [2]:
# STEP 3
class Resource:
    def __init__(self, name, manufacturer, total, allocated):
        self._name = name
        self._manufacturer = manufacturer

        validate_integer('total', total, min_value=0)
        self._total = total

        validate_integer(
            'allocated', allocated, 0, total,
            custom_max_message='Allocated inventory cannot exceed total inventory'
        )
        self._allocated = allocated

    @property
    def name(self):
        return self._name

    @property
    def manufacturer(self):
        return self._manufacturer

    @property
    def total(self):
        return self._total

    @property
    def allocated(self):
        return self._allocated

    @property
    def category(self):
        return type(self).__name__.lower()

    @property
    def available(self):
        return self.total - self.allocated

    def __str__(self):
        return self.name

    def __repr__(self):
        return (f'{self.name} ({self.category} - {self.manufacturer}) : '
                f'total={self.total}, allocated={self.allocated}'
                )

    def claim(self, num):
        validate_integer(
            'num', num, 1, self.available,
            custom_max_message='Cannot claim more than available'
        )
        self._allocated += num

    def freeup(self, num):
        validate_integer(
            'num', num, 1, self.allocated,
            custom_max_message='Cannot return more than allocated'
        )
        self._allocated -= num

    def died(self, num):
        validate_integer('num', num, 1, self.allocated,
                         custom_max_message='Cannot retire more than allocated')
        self._total -= num
        self._allocated -= num

    def purchased(self, num):
        validate_integer('num', num, 1)
        self._total += num

In [3]:
# STEP 5
class CPU(Resource):
    def __init__(
            self, name, manufacturer, total, allocated,
            cores, socket, power_watts
    ):
        super().__init__(name, manufacturer, total, allocated)

        validate_integer('cores', cores, 1)
        validate_integer('power_watts', power_watts, 1)

        self._cores = cores
        self._socket = socket
        self._power_watts = power_watts

    @property
    def cores(self):
        return self._cores

    @property
    def socket(self):
        return self._socket

    @property
    def power_watts(self):
        return self._power_watts

    def __repr__(self):
        return f'{self.category}: {self.name} ({self.socket} - x{self.cores})'


class Storage(Resource):
    def __init__(self, name, manufacturer, total, allocated, capacity_gb):
        
        super().__init__(name, manufacturer, total, allocated)
        validate_integer('capacity_gb', capacity_gb, 1)
        self._capacity_gb = capacity_gb

    @property
    def capacity_gb(self):
        return self._capacity_gb

    def __repr__(self):
        return f'{self.category}: {self.capacity_gb} GB'


class HDD(Storage):
    def __init__(
            self, name, manufacturer, total, allocated, capacity_gb,
            size, rpm
    ):
        super().__init__(name, manufacturer, total, allocated, capacity_gb)

        allowed_sizes = ['2.5"', '3.5"']
        if size not in allowed_sizes:
            raise ValueError(f'Invalid HDD size. '
                             f'Must be one of {", ".join(allowed_sizes)}')
        validate_integer('rpm', rpm, min_value=1_000, max_value=50_000)

        self._size = size
        self._rpm = rpm

    @property
    def size(self):
        return self._size

    @property
    def rpm(self):
        return self._rpm

    def __repr__(self):
        s = super().__repr__()
        return f'{s} ({self.size}, {self.rpm} rpm)'


class SSD(Storage):
    def __init__(
            self, name, manufacturer, total, allocated, capacity_gb,
            interface
    ):
        super().__init__(name, manufacturer, total, allocated, capacity_gb)

        self._interface = interface

    @property
    def interface(self):
        return self._interface

    def __repr__(self):
        s = super().__repr__()
        return f'{s} ({self.interface})'

#### Tests Solution

In [5]:
# STEP 2

In [6]:
import pytest
import ipytest
ipytest.autoconfig()

##### Validator

In [9]:
class TestIntegerValidator:
    def test_valid(self):
        validate_integer('arg', 10, 0, 20, 'custom min msg', 'custom max msg')

    def test_type_error(self):
        with pytest.raises(TypeError):
            validate_integer('arg', 1.5)

    def test_min_std_err_msg(self):
        with pytest.raises(ValueError) as ex:
            validate_integer('arg', 10, 100)
        assert 'arg' in str(ex.value)
        assert '100' in str(ex.value)

    def test_min_custom_msg(self):
        with pytest.raises(ValueError) as ex:
            validate_integer('arg', 10, 100, custom_min_message='custom')
        assert str(ex.value) == 'custom'

    def test_max_std_err_msg(self):
        with pytest.raises(ValueError) as ex:
            validate_integer('arg', 10, 1, 5)
        assert 'arg' in str(ex.value)
        assert '5' in str(ex.value)

    def test_max_custom_err_msg(self):
        with pytest.raises(ValueError) as ex:
            validate_integer('arg', 10, 1, 5, custom_max_message='custom')
        assert str(ex.value) == 'custom'

##### Resource

Pytest fixtures are a flexible and powerful way to manage test setup and teardown, making your tests cleaner and more maintainable.

The yield keyword is used to separate setup code from teardown code. Code before the yield runs before the test, and code after the yield runs after the test.


To explain one example below: we apply the pytest fixture on the `resource_values` function marking it as a fixture which will be passed as an argument to one of our test functions.

When this test function runs, it will call (inject) the fixture into the test function making whatever was returned from the fixture available to the test function.

`test_create_resource` has the `resource` fixture as an argument. This will be called to return a `Resource` object. This object itself will call the `resource_values` fixture to populate the `Resource` parameters. 

In [10]:
# STEP 4
@pytest.fixture
def resource_values():
    return {
        'name': 'Parrot',
        'manufacturer': 'Pirates A-Hoy',
        'total': 100,
        'allocated': 50
    }


@pytest.fixture
def resource(resource_values):
    return Resource(**resource_values)


def test_create_resource(resource_values, resource):
    for attr_name in resource_values:
        assert getattr(resource, attr_name) == resource_values.get(attr_name)


def test_create_invalid_total_type():
    with pytest.raises(TypeError):
        Resource('Parrot', 'Pirates A-Hoy', 10.5, 5)


def test_create_invalid_allocated_type():
    with pytest.raises(TypeError):
        Resource('name', 'manu', 10, 2.5)


def test_create_invalid_total_value():
    with pytest.raises(ValueError):
        Resource('name', 'manu', -10, 0)


`parametrize` allows us to call the same test function but with various arguments. 

In the first example below, we want to create a resource whose `total=10` and `allocated=-5`. 

Then, we want to create a resource whose `total=10` and `allocated=20`. 

We can do this in one shot with `parametrize`:

For the first argument, we supply the names of these variable arguments as a comma-separated string.

For the second argument, we supply an iterable of subiterables, where the subiterables are the argument pair to be unpacked as `total` and `allocated` in that order.

In [11]:
@pytest.mark.parametrize('total,allocated', [(10, -5), (10, 20)])
def test_create_invalid_allocated_value(total, allocated):
    with pytest.raises(ValueError):
        Resource('name', 'manu', total, allocated)


def test_total(resource):
    assert resource.total == resource._total


def test_allocated(resource):
    assert resource.allocated == resource._allocated


def test_available(resource, resource_values):
    assert resource.available == resource.total - resource.allocated


def test_category(resource):
    assert resource.category == 'resource'


def test_str_repr(resource):
    assert str(resource) == resource.name


def test_repr_repr(resource):
    assert repr(resource) == '{} ({} - {}) : total={}, allocated={}'.format(
        resource.name, resource.category, resource.manufacturer, resource.total,
        resource.allocated
    )


def test_claim(resource):
    n = 2
    original_total = resource.total
    original_allocated = resource.allocated
    resource.claim(n)
    assert resource.total == original_total
    assert resource.allocated == original_allocated + n


@pytest.mark.parametrize('value', [-1, 0, 1_000])
def test_claim_invalid(resource, value):
    with pytest.raises(ValueError):
        resource.claim(value)


def test_freeup(resource):
    n = 2
    original_total = resource.total
    original_allocated = resource.allocated
    resource.freeup(n)
    assert resource.allocated == original_allocated - n
    assert resource.total == original_total


@pytest.mark.parametrize('value', [-1, 0, 1_000])
def test_freeup_invalid(resource, value):
    with pytest.raises(ValueError):
        resource.freeup(value)


def test_died(resource):
    n = 2
    original_total = resource.total
    original_allocated = resource.allocated
    resource.died(n)
    assert resource.total == original_total - n
    assert resource.allocated == original_allocated - n


@pytest.mark.parametrize('value', [-1, 0, 1_000])
def test_died_invalid(resource, value):
    with pytest.raises(ValueError):
        resource.died(value)


def test_purchased(resource):
    n = 2
    original_total = resource.total
    original_allocated = resource.allocated
    resource.purchased(n)
    assert resource.total == original_total + n
    assert resource.allocated == original_allocated


@pytest.mark.parametrize('value', [-1, 0])
def test_purchased_invalid(resource, value):
    with pytest.raises(ValueError):
        resource.purchased(value)

##### CPU

In [12]:
# STEP 6
@pytest.fixture
def cpu_values():
    return {
        'name': 'RYZEN Threadripper 2990WX',
        'manufacturer': 'AMD',
        'total': 10,
        'allocated': 3,
        'cores': 32,
        'socket': 'sTR4',
        'power_watts': 250
    }


@pytest.fixture
def cpu(cpu_values):
    return CPU(**cpu_values)


def test_create_cpu(cpu, cpu_values):
    for attr_name in cpu_values:
        assert getattr(cpu, attr_name) == cpu_values.get(attr_name)


@pytest.mark.parametrize(
    'cores, exception', [(10.5, TypeError), (-1, ValueError), (0, ValueError)]
)
def test_create_invalid_cores(cores, exception, cpu_values):
    cpu_values['cores'] = cores
    with pytest.raises(exception):
        CPU(**cpu_values)


@pytest.mark.parametrize(
    'watts, exception', [(10.5, TypeError), (-1, ValueError), (0, ValueError)]
)
def test_create_invalid_power(watts, exception, cpu_values):
    cpu_values['power_watts'] = watts
    with pytest.raises(exception):
        CPU(**cpu_values)


def test_repr(cpu):
    assert cpu.category in repr(cpu)
    assert cpu.name in repr(cpu)
    assert cpu.socket in repr(cpu)
    assert str(cpu.cores) in repr(cpu)

##### Storage

In [13]:
@pytest.fixture
def storage_values():
    return {
        'name': 'Thumbdrive',
        'manufacturer': 'Sandisk',
        'total': 10,
        'allocated': 3,
        'capacity_gb': 512
    }


@pytest.fixture
def storage(storage_values):
    return Storage(**storage_values)


def test_create(storage, storage_values):
    for attr_name in storage_values:
        assert getattr(storage, attr_name) == storage_values.get(attr_name)


@pytest.mark.parametrize(
    'gb, exception', [(10.5, TypeError), (-1, ValueError), (0, ValueError)]
)
def test_create_invalid_storage(gb, exception, storage_values):
    storage_values['capacity_gb'] = gb
    with pytest.raises(exception):
        Storage(**storage_values)


def test_repr(storage):
    assert storage.category in repr(storage)
    assert str(storage.capacity_gb) in repr(storage)

##### SSD

In [7]:
@pytest.fixture
def ssd_values():
    return {
        'name': 'Samsung 860 EVO',
        'manufacturer': 'Samsung',
        'total': 10,
        'allocated': 3,
        'capacity_gb': 1_000,
        'interface': 'SATA III'
    }


@pytest.fixture
def ssd(ssd_values):
    return SSD(**ssd_values)


def test_create(ssd, ssd_values):
    for attr_name in ssd_values:
        assert getattr(ssd, attr_name) == ssd_values.get(attr_name)


def test_repr(ssd):
    assert ssd.category in repr(ssd)
    assert str(ssd.capacity_gb) in repr(ssd)
    assert ssd.interface in repr(ssd)

##### HDD

In [14]:
@pytest.fixture
def hdd_values():
    return {
        'name': '1TB SATA HDD',
        'manufacturer': 'Seagate',
        'total': 10,
        'allocated': 3,
        'capacity_gb': 1_000,
        'size': '3.5"',
        'rpm': 10_000
    }


@pytest.fixture
def hdd(hdd_values):
    return HDD(**hdd_values)


def test_create(hdd, hdd_values):
    for attr_name in hdd_values:
        assert getattr(hdd, attr_name) == hdd_values.get(attr_name)


@pytest.mark.parametrize('size', ['2.5', '5.25"'])
def test_create_invalid_size(size, hdd_values):
    hdd_values['size'] = size
    with pytest.raises(ValueError):
        HDD(**hdd_values)


@pytest.mark.parametrize(
    'rpm, exception',
    [
        ('100', TypeError),
        (100, ValueError),
        (100_000, ValueError)
    ]
)
def test_create_invalid_rpm(rpm, exception, hdd_values):
    hdd_values['rpm'] = rpm
    with pytest.raises(exception):
        HDD(**hdd_values)


def test_repr(hdd):
    assert hdd.category in repr(hdd)
    assert str(hdd.capacity_gb) in repr(hdd)
    assert hdd.size in repr(hdd)
    assert str(hdd.rpm) in repr(hdd)

##### Running the tests

In [15]:
ipytest.run()

..................................................                                           [100%]
50 passed in 0.15s


<ExitCode.OK: 0>